# 🧹 Workshop Cleanup - Resource Deletion

**IMPORTANT**: Run this notebook after completing all workshop sessions to avoid ongoing charges.

This notebook will help you clean up all AWS resources created during the Mistral AI Workshop:

1. **Delete Mistral Small 3 Bedrock Marketplace Endpoint**
2. **Delete Mistral OCR SageMaker Endpoint** 
3. **Manual cleanup instructions** for SageMaker Studio and CloudFormation stack

## Step 1: Import Required Libraries

In [1]:
import boto3
import time
from botocore.exceptions import ClientError

# Initialize AWS clients
bedrock_client = boto3.client('bedrock', region_name='us-west-2')
sagemaker_client = boto3.client('sagemaker', region_name='us-west-2')

print("✅ AWS clients initialized")
print(f"🌍 Region: us-west-2")

✅ AWS clients initialized
🌍 Region: us-west-2


## Step 2: List and Delete Bedrock Marketplace Endpoints

This will find and delete Mistral Small 3 endpoints created through Bedrock Marketplace.

In [2]:
def list_bedrock_marketplace_endpoints():
    """List all Bedrock marketplace endpoints"""
    try:
        response = bedrock_client.list_marketplace_model_endpoints()
        endpoints = response.get('marketplaceModelEndpoints', [])
        
        if not endpoints:
            print("ℹ️ No Bedrock marketplace endpoints found")
            return []
        
        print(f"📋 Found {len(endpoints)} Bedrock marketplace endpoint(s):")
        for endpoint in endpoints:
            arn = endpoint['endpointArn']
            status = endpoint['status']
            print(f"     ARN: {arn}")
            print(f"   • {status}")
        
        return endpoints
    
    except ClientError as e:
        print(f"❌ Error listing Bedrock endpoints: {e}")
        return []

# List all Bedrock marketplace endpoints
bedrock_endpoints = list_bedrock_marketplace_endpoints()

📋 Found 1 Bedrock marketplace endpoint(s):
     ARN: arn:aws:sagemaker:us-west-2:459006231907:endpoint/endpoint-quick-start-vb47a
   • REGISTERED


In [3]:
def delete_bedrock_marketplace_endpoint(endpoint_arn):
    """Delete a specific Bedrock marketplace endpoint"""
    try:
        print(f"🗑️ Deleting Bedrock endpoint: {endpoint_arn}")
        
        response = bedrock_client.delete_marketplace_model_endpoint(
            endpointArn=endpoint_arn
        )
        
        print(f"✅ Deletion initiated for: {endpoint_arn}")
        print(f"   Status will change to 'Deleting' then 'Deleted'")
        
        return True
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceNotFoundException':
            print(f"ℹ️ Endpoint {endpoint_arn} not found (may already be deleted)")
        else:
            print(f"❌ Error deleting endpoint {endpoint_arn}: {e}")
        return False

In [4]:
# Delete endpoint from the list 
endpoint_arn = "arn:aws:sagemaker:us-west-2:459006231907:endpoint/endpoint-quick-start-vb47a"

delete_bedrock_marketplace_endpoint(endpoint_arn)

🗑️ Deleting Bedrock endpoint: arn:aws:sagemaker:us-west-2:459006231907:endpoint/endpoint-quick-start-vb47a
✅ Deletion initiated for: arn:aws:sagemaker:us-west-2:459006231907:endpoint/endpoint-quick-start-vb47a
   Status will change to 'Deleting' then 'Deleted'


True

## Step 3: List and Delete SageMaker Endpoints

This will find and delete Mistral OCR endpoints and any other SageMaker endpoints created during the workshop.

In [5]:
def list_sagemaker_endpoints():
    """List all SageMaker endpoints"""
    try:
        response = sagemaker_client.list_endpoints()
        endpoints = response.get('Endpoints', [])
    
        if not endpoints:
            print("ℹ️ No SageMaker endpoints found")
            return []
        
        print(f"📋 Found {len(endpoints)} SageMaker endpoint(s):")
        for endpoint in endpoints:
            print(endpoint)
            name = endpoint['EndpointName']
            status = endpoint['EndpointStatus']
            instance_type = endpoint.get('InstanceType', 'Unknown')
            creation_time = endpoint['CreationTime']
            print(f"   • {name} ({status})")
            print(f"     Instance: {instance_type} | Created: {creation_time}")
        
        return endpoints
    
    except ClientError as e:
        print(f"❌ Error listing SageMaker endpoints: {e}")
        return []

# List all SageMaker endpoints
sagemaker_endpoints = list_sagemaker_endpoints()

📋 Found 1 SageMaker endpoint(s):
{'EndpointName': 'mistral-ocr-endpoint', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:459006231907:endpoint/mistral-ocr-endpoint', 'CreationTime': datetime.datetime(2025, 6, 18, 10, 30, 32, 546000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2025, 6, 18, 10, 36, 28, 994000, tzinfo=tzlocal()), 'EndpointStatus': 'InService'}
   • mistral-ocr-endpoint (InService)
     Instance: Unknown | Created: 2025-06-18 10:30:32.546000+00:00


In [6]:
def delete_sagemaker_endpoint(endpoint_name):
    """Delete a SageMaker endpoint and its configuration"""
    try:
        print(f"🗑️ Deleting SageMaker endpoint: {endpoint_name}")
        
        # Get endpoint configuration name
        endpoint_desc = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
        config_name = endpoint_desc['EndpointConfigName']
        
        # Delete endpoint
        sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
        print(f"✅ Endpoint deletion initiated: {endpoint_name}")
        
        # Delete endpoint configuration
        try:
            sagemaker_client.delete_endpoint_config(EndpointConfigName=config_name)
            print(f"✅ Endpoint config deleted: {config_name}")
        except ClientError as config_error:
            print(f"⚠️ Could not delete endpoint config {config_name}: {config_error}")
        
        return True
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ValidationException':
            print(f"ℹ️ Endpoint {endpoint_name} not found (may already be deleted)")
        else:
            print(f"❌ Error deleting endpoint {endpoint_name}: {e}")
        return False


In [7]:
# Delete SageMaker endpoint that created in the Workshop
endpoint_name = "mistral-ocr-endpoint"
delete_sagemaker_endpoint(endpoint_name)

🗑️ Deleting SageMaker endpoint: mistral-ocr-endpoint
✅ Endpoint deletion initiated: mistral-ocr-endpoint
✅ Endpoint config deleted: endpointConfig-1750242612187


True

## Step 5: Manual Cleanup Instructions

⚠️ **IMPORTANT**: Complete these final steps manually to fully clean up your workshop environment.

### 4.1 Delete SageMaker Studio Space

1. **Go to SageMaker Console**:
   - Navigate to [SageMaker Console](https://us-west-2.console.aws.amazon.com/sagemaker/home?region=us-west-2#/studio) in us-west-2
   - Click on **"Studio"** in the left sidebar

2. **Access Space Management**:
   - Click on your Domain name
   - Go to **"Space management"** tab
   - Find your workshop space (usually named after your user)

3. **Delete the Space**:
   - Select your space
   - Click **"Delete"**
   - Confirm deletion
   - Wait for deletion to complete (may take 5-10 minutes)

### 4.2 Delete CloudFormation Stack

1. **Go to CloudFormation Console**:
   - Navigate to [CloudFormation Console](https://us-west-2.console.aws.amazon.com/cloudformation/home?region=us-west-2#/stacks) in us-west-2

2. **Find Workshop Stack**:
   - Look for stack named similar to:
     - `mistral-workshop-*`

3. **Delete the Stack**:
   - Select the workshop stack
   - Click **"Delete"**
   - Confirm deletion
   - **Wait for completion**

------

## 🎉 Workshop Complete!

Thank you for participating in the Mistral AI Workshop! 

**Remember**: Always clean up cloud resources after workshops to avoid unexpected charges! 💸